In [1]:
import pandas as pd
import numpy as np
import os.path as osp
import re
%load_ext dotenv
%dotenv env.sh
%run -m cytokit_nb.keyence

In [2]:
df = pd.read_csv('experiments-base.csv')
df

,name,dir
0,00.0mT-np-0uMsta,untreated-noTcells-2
1,02.5mT-np-0uMsta,025mioTcells-nopeptide
2,05.0mT-np-0uMsta,5mioTcells-nopeptide
3,10.0mT-np-0uMsta,10mioTcells-nopeptide
4,20.0mT-np-0uMsta,20mioTcells-nopeptide
5,02.5mT-wp-0uMsta,025mioTcells-peptide
6,05.0mT-wp-0uMsta,5mioTcells-peptide
7,10.0mT-wp-0uMsta,10mioTcells-peptide
8,20.0mT-wp-0uMsta,20mioTcells-peptide
9,{:04.1f}mT-wp-0uMsta,20-10-5mioTcells-peptide


In [3]:
grids = ['XY01', 'XY02', 'XY03']


def get_row(r, grid, info, base_conf, rep):
    return r.append(pd.Series({
        'grid': grid,
        'rep': rep, 
        'z_pitch': info['z_pitch'].iloc[0],
        'n_tiles': info['tile'].nunique(),
        'n_z': info['z'].nunique(),
        'n_ch': info['ch'].nunique(),
        'chs': tuple(sorted(info['ch'].unique())),
        'conf': base_conf
    })).sort_index()

def add_info(r, grid):
    r = r.copy()
    path = osp.join(os.environ['EXP_GROUP_RAW_DIR'], r['dir'], grid)    
    info = analyze_keyence_dataset(path)
    base_conf = 'experiment.yaml'
    rep = {'XY01': 1, 'XY02': 2, 'XY03': 3}[grid]
    if re.search(r'\{.*\}', r['name']):
        if r['dir'] != '20-10-5mioTcells-peptide':
            raise ValueError('Invalid row {}'.format(r))
        # Map this one case onto concentrations/conditions based on grid (all others are reps)
        fmt = r['name']
        r['name'] = {
            'XY01': '20.0mT-wp-0uMsta',
            'XY02': '10.0mT-wp-0uMsta',
            'XY03': '05.0mT-wp-0uMsta'
        }[grid]
        rep = 4
    return get_row(r, grid, info, base_conf, rep)

dfi = pd.DataFrame([
    add_info(r, grid=g)
    for _, r in df.iterrows()
    for g in grids
])
dfi.head()

,chs,conf,dir,grid,n_ch,n_tiles,n_z,name,rep,z_pitch
0,"(1, 2, 3, 4)",experiment.yaml,untreated-noTcells-2,XY01,4,33,8,00.0mT-np-0uMsta,1,200
1,"(1, 2, 3, 4)",experiment.yaml,untreated-noTcells-2,XY02,4,33,8,00.0mT-np-0uMsta,2,200
2,"(1, 2, 3, 4)",experiment.yaml,untreated-noTcells-2,XY03,4,33,7,00.0mT-np-0uMsta,3,200
3,"(1, 2, 3, 4)",experiment.yaml,025mioTcells-nopeptide,XY01,4,33,9,02.5mT-np-0uMsta,1,200
4,"(1, 2, 3, 4)",experiment.yaml,025mioTcells-nopeptide,XY02,4,33,8,02.5mT-np-0uMsta,2,200


In [4]:
dfi['n_tiles'].unique()

array([33])

In [5]:
dfi['z_pitch'].unique()

array([200])

In [6]:
dfi['chs'].unique()

array([('1', '2', '3', '4')], dtype=object)

In [7]:
assert dfi['chs'].nunique() == 1
assert dfi['n_ch'].nunique() == 1
assert dfi['z_pitch'].nunique() == 1
assert dfi['n_tiles'].nunique() == 1

In [9]:
dfe = dfi.copy()
dfe = dfe.rename(columns={'name': 'cond'})
dfe = dfe.drop('chs', axis=1)
dfe.insert(0, 'name', dfe.apply(lambda r: '{}-{}-{}'.format(r['cond'], r['grid'], r['rep']), axis=1))
dfe.head()

,name,conf,dir,grid,n_ch,n_tiles,n_z,cond,rep,z_pitch
0,00.0mT-np-0uMsta-XY01-1,experiment.yaml,untreated-noTcells-2,XY01,4,33,8,00.0mT-np-0uMsta,1,200
1,00.0mT-np-0uMsta-XY02-2,experiment.yaml,untreated-noTcells-2,XY02,4,33,8,00.0mT-np-0uMsta,2,200
2,00.0mT-np-0uMsta-XY03-3,experiment.yaml,untreated-noTcells-2,XY03,4,33,7,00.0mT-np-0uMsta,3,200
3,02.5mT-np-0uMsta-XY01-1,experiment.yaml,025mioTcells-nopeptide,XY01,4,33,9,02.5mT-np-0uMsta,1,200
4,02.5mT-np-0uMsta-XY02-2,experiment.yaml,025mioTcells-nopeptide,XY02,4,33,8,02.5mT-np-0uMsta,2,200


In [10]:
dfe

,name,conf,dir,grid,n_ch,n_tiles,n_z,cond,rep,z_pitch
0,00.0mT-np-0uMsta-XY01-1,experiment.yaml,untreated-noTcells-2,XY01,4,33,8,00.0mT-np-0uMsta,1,200
1,00.0mT-np-0uMsta-XY02-2,experiment.yaml,untreated-noTcells-2,XY02,4,33,8,00.0mT-np-0uMsta,2,200
2,00.0mT-np-0uMsta-XY03-3,experiment.yaml,untreated-noTcells-2,XY03,4,33,7,00.0mT-np-0uMsta,3,200
3,02.5mT-np-0uMsta-XY01-1,experiment.yaml,025mioTcells-nopeptide,XY01,4,33,9,02.5mT-np-0uMsta,1,200
4,02.5mT-np-0uMsta-XY02-2,experiment.yaml,025mioTcells-nopeptide,XY02,4,33,8,02.5mT-np-0uMsta,2,200
5,02.5mT-np-0uMsta-XY03-3,experiment.yaml,025mioTcells-nopeptide,XY03,4,33,8,02.5mT-np-0uMsta,3,200
6,05.0mT-np-0uMsta-XY01-1,experiment.yaml,5mioTcells-nopeptide,XY01,4,33,6,05.0mT-np-0uMsta,1,200
7,05.0mT-np-0uMsta-XY02-2,experiment.yaml,5mioTcells-nopeptide,XY02,4,33,7,05.0mT-np-0uMsta,2,200
8,05.0mT-np-0uMsta-XY03-3,experiment.yaml,5mioTcells-nopeptide,XY03,4,33,7,05.0mT-np-0uMsta,3,200
9,10.0mT-np-0uMsta-XY01-1,experiment.yaml,10mioTcells-nopeptide,XY01,4,33,7,10.0mT-np-0uMsta,1,200


In [11]:
dfe.to_csv('experiments.csv', index=False)